<a href="https://colab.research.google.com/github/Saran-nns/incremental_learning_tf2.0/blob/master/incremental_learning_model_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

### Data

In [2]:
dataset = datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
def custom_dataset_loader(dataset = dataset, new_data = None, old_labels=[0,1,2,3,4,5,6,7], new_labels= [8,9]):
  
  filtered_training_data = []
  filtered_labels = []
  allowed_labels = old_labels + new_labels
  
  # Pick only the data with required labels
  for i, _ in enumerate(dataset[0][0]):
    if dataset[0][1][i] in allowed_labels:
      filtered_training_data.append(dataset[0][0][i]/np.float32(255))
      filtered_labels.append(dataset[0][1][i])

  train_dataset = tf.data.Dataset.from_tensor_slices((filtered_training_data, filtered_labels))
  # TODO: new_dataset = train_dataset+old_dataset
  train_dataset.shuffle(len(list(train_dataset)))

  # Split into train and test (80%, 20%)
  trainind_dataset_size = int(len(list(train_dataset))*0.8)
  train_ds,test_ds = train_dataset.take(trainind_dataset_size), train_dataset.skip(trainind_dataset_size)

  # Split the images and labels in train and test datasets
  train_images,train_labels= np.array(list(train_ds))[:,0],np.array(list(train_ds))[:,1]
  train_images,train_labels = tf.convert_to_tensor(train_images.tolist()), tf.convert_to_tensor(train_labels.tolist())
  test_images,test_labels = np.array(list(test_ds))[:,0],np.array(list(test_ds))[:,1]
  test_images,test_labels = tf.convert_to_tensor(test_images.tolist()),tf.convert_to_tensor(test_labels.tolist())
  print('Shape of training images after filtering labels',np.shape(train_images))

  return train_images,train_labels,test_images,test_labels

### Model

##### Conditions for adding new layers

In [0]:
class FeatureExtractor(Model):
  def __init__(self,trainable):
    super(FeatureExtractor, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu',trainable = trainable)
    self.maxpool = MaxPooling2D((2,2))
    self.conv2 = Conv2D(64, 3, activation='relu',trainable = trainable)
    self.conv3 = Conv2D(64,3,activation='relu',trainable = trainable)
  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool(x)
    x = self.conv2(x)
    x = self.maxpool(x)
    x = self.conv3(x)
    return x

class Classifier(Model):
  def __init__(self, n_classes):
    super(Classifier, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(64, activation='relu')
    self.d2 = Dense(n_classes)

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

class IncrementalLearning(Model):
  def __init__(self, feature_extractor,classifier = None, is_online = False, add_new_prediction_layer = False):
    super(IncrementalLearning, self).__init__()
    self.feature_extractor=feature_extractor
    self.classifier = classifier
    if is_online:
      self.feature_extractor.trainable = False

    if classifier is not None:
      # Get the layers of the classifier
      layer_names = [var.name for var in classifier.trainable_variables]
      if not layer_names:

      if add_new_prediction_layer:
        self.classifier.trainable = False
        # Create new layer inside classifier model
        
    

  def call(self, x):
    features = self.feature_extractor(x)
    x = self.classifier(features)

    return x

In [17]:
x = Classifier(10)
x.build((64,))
var = [var.name for var in x.trainable_variables]
var

['dense_20/kernel:0',
 'dense_20/bias:0',
 'dense_21/kernel:0',
 'dense_21/bias:0']

In [0]:
model = IncrementalLearning(feature_extractor,classifier)